In [174]:
import pandas as pd
import os
from glob import glob
import numpy as np

In [175]:
all_files = [file for path, subdir, files in os.walk('data/HistoricData') for file in glob(os.path.join(path, '*.CSV'))]
all_files_232 = []
for filename in all_files:
    if '232' in filename:
        all_files_232.append(filename)
all_files_232, len(all_files_232)

(['data/HistoricData/PST232/PST232_2021_Juli.CSV',
  'data/HistoricData/PST232/PST232_2020_Juli.CSV',
  'data/HistoricData/PST232/PST232_2021_August.CSV',
  'data/HistoricData/PST232/PST232_2021_Marts.CSV',
  'data/HistoricData/PST232/PST232_2020_Juni.CSV',
  'data/HistoricData/PST232/PST232_2021_Juni.CSV',
  'data/HistoricData/PST232/PST232_2020_November.CSV',
  'data/HistoricData/PST232/PST232_2020_December.CSV',
  'data/HistoricData/PST232/PST232_2020_Oktober.CSV',
  'data/HistoricData/PST232/PST232_2021_Februar.CSV',
  'data/HistoricData/PST232/PST232_2020_September.CSV',
  'data/HistoricData/PST232/PST232_2020_Maj.CSV',
  'data/HistoricData/PST232/PST232_2020_September_2.CSV',
  'data/HistoricData/PST232/PST232_2021_Januar.CSV',
  'data/HistoricData/PST232/PST232_2021_April.CSV',
  'data/HistoricData/PST232/PST232_2021_Maj.CSV',
  'data/HistoricData/PST232/PST232_2020_August.CSV'],
 17)

In [178]:
p = all_files_232[0]
print(f'reading {p}')
df = pd.read_csv(p, encoding='cp1252', sep=';', decimal=',').reset_index().iloc[:,1:]
df

reading data/HistoricData/PST232/PST232_2020_Juli.CSV


,Tid,PST-232-P1-Strøm Senest målte motorstrøm P1 (0.0-20.0 A),PST-232-P2-Strøm Senest målte motorstrøm P2 (0.0-20.0 A),PST-232-Niveau Niveau (0.00-10.00 m),PST-232-Flow_ud Flow (0.0-250.0 m3/h)
0,"00:00,0",0.0,0.0,0.96,0.0
1,"00:51,1",0.0,0.0,0.97,0.0
2,"03:14,4",0.0,0.0,0.98,0.0
3,"03:29,1",0.0,0.0,0.97,0.0
4,"03:44,5",0.0,0.0,0.98,0.0
...,...,...,...,...,...
146079,"55:37,2",0.0,0.0,0.35,0.0
146080,"56:53,7",0.0,0.0,0.36,0.0
146081,"57:29,2",0.0,0.0,0.37,0.0
146082,"58:20,1",0.0,0.0,0.38,0.0


In [179]:
df.columns = ['time', 'current_1', 'current_2', 'water_level', 'outflow_level']
df = df.astype({
    'current_1': float,
    'current_2': float,
    'water_level': float,
    'outflow_level': float,
})
df.time = pd.to_datetime(df.time)
df

ParserError: hour must be in 0..23: 25:25,3

In [163]:
df['currents'] = df.apply(lambda row: [row.current_1, row.current_2], axis=1)
df['current_tot'] = df.apply(lambda row: row.current_1 + row.current_2, axis=1)
df.drop(columns=['current_1', 'current_2'], inplace=True)

In [170]:
df_subset = df[0:]
df_subset = df_subset.set_index(df_subset.time)
df_subset.drop(columns=['time','currents'], inplace=True)
df_subset

,water_level,outflow_level,current_tot
time,,,
2021-01-07 00:00:00.000,1.05,0.0,0.0
2021-01-07 00:01:26.166,1.07,0.0,0.0
2021-01-07 00:04:18.640,1.09,0.0,0.0
2021-01-07 00:07:34.170,1.11,0.0,0.0
2021-01-07 00:11:12.242,1.13,0.0,0.0
...,...,...,...
2021-07-31 23:57:16.738,0.26,0.0,0.0
2021-07-31 23:57:39.732,0.27,0.0,0.0
2021-07-31 23:58:37.031,0.28,0.0,0.0


In [171]:
def resample_time_weighted_mean(x, target_index, closed=None, label=None):
    shift = 1 if closed == "right" else -1
    fill = "bfill" if closed == "right" else "ffill"
    # Determine length of each interval (daylight saving aware)
    extended_index = target_index.union(
        [target_index[0] - target_index.freq, target_index[-1] + target_index.freq]
    )
    interval_lengths = -extended_index.to_series().diff(periods=shift)

    # Create a combined index of the source index and target index and reindex to combined index
    combined_index = x.index.union(extended_index)
    x = x.reindex(index=combined_index, method=fill)
    interval_lengths = interval_lengths.reindex(index=combined_index, method=fill)

    # Determine weights of each value and multiply source values
    weights = -x.index.to_series().diff(periods=shift) / interval_lengths
    x = x.mul(weights, axis=0)

    # Resample to new index, the final reindex is necessary because resample
    # might return more rows based on the frequency
    return (
        x.resample(target_index.freq, closed=closed, label=label)
        .sum()
        .reindex(target_index)
    )

In [172]:


opts = dict(closed="left", label="left")

resample_time_weighted_mean(
    df_subset, pd.DatetimeIndex(df_subset.resample("10S", **opts).groups.keys(), freq="infer"), **opts
)

ValueError: index must be monotonic increasing or decreasing